In [1]:
from deepface import DeepFace
import face_recognition
from PIL import Image
import os
from os import listdir
#from multiprocessing import Pool #mission impossible currently

In [2]:
import torch
import torchvision.transforms as transforms
import numpy as np
from numpy import asarray
import torch.nn.functional as F
from emotion_detection.fer_data_utils import SkResize, HistEq, AddChannel, ToRGB
from vision_utils.custom_architectures import SepConvModel
#extra added
from vision_utils.custom_architectures import SepConvModel, initialize_model, PretrainedMT
from multitask_rag.evaluate import predict_utk as test_image

In [3]:
import pandas as pd
import urllib
import requests

import certifi
import ssl
import gc

In [4]:
#for the downloading of images from the link
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 
context = ssl._create_unverified_context()

In [5]:
PATHSEPCONV='/home/manish//Documents/democlassi/emotion-detection/sepconv_model_55_val_loss=1.175765.pth'
PATHRESNETAGR='/home/manish/Documents/democlassi/other/resnet_model_21_val_loss=4.275671.pth'

In [6]:
#emotion
model = SepConvModel()
model.load_state_dict(torch.load(PATHSEPCONV, map_location="cpu"))

#age-race-gender
resnet_model_agr = PretrainedMT(model_name='resnet')
resnet_model_agr.load_state_dict(torch.load(PATHRESNETAGR, map_location="cpu"))

<All keys matched successfully>

In [7]:
data_transforms = transforms.Compose([
    #transforms.Resize((500,500),Image.Resampling.LANCZOS),
    transforms.Resize((200,200),Image.Resampling.LANCZOS),
    transforms.ToTensor()
])

In [8]:
def democlassiEvaluate(pil_image,):
    
    img_tensor= data_transforms(pil_image) 
    imge = pil_image.convert('L')
    img = imge.resize((120,120),Image.Resampling.LANCZOS)
    numpydata = asarray(img)
    res1=predict_fer(numpydata,model,True)
    # with resent model--age--race--gender
    #res2=test_image(img_tensor,resnet_model)
    #with sep_conv_model --age--gender---race
    #res2=test_image(img_tensor,sep_conv_model)
    #with VGG model ---age--race--gender
    #res2=test_image(img_tensor,vgg_model_agr)
    
    #for resnet model
    res2 = test_image(img_tensor,resnet_model_agr)
    
    #for sepconv_ model
    #res2 = test_image(img_tensor,sepconv_model_agr)
    return res1,res2 

In [9]:
def deepfaceOutput(obj1):
    print("Deepface comparision")
    print("----Deepface---")
    print("Age:---",{obj1['age']},"---")
    print("Gender:---",{obj1['gender']},"---")
    print("Race:---",{obj1['dominant_race']},"---")
    print("Emotion:---",{obj1['dominant_emotion']},"---")
    print('------------------------------------------- ')


In [10]:
def predict_fer(image, model, transf_learn=True):

    # process image
    image = preprocess_fer(image, transf_learn)
    
    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    emotion = F.softmax(model(image), dim=1).detach().to('cpu').numpy()[0]
    target_names = ['Angry', 'Disgusted', 'Afraid', 'Happy', 'Sad', 'Surprised', 'Neutral']
    pred_label = target_names[np.argmax(emotion)]

    emotion_probs = dict(zip(target_names, emotion))

    return emotion_probs, pred_label

In [11]:
def preprocess_fer(image, transf_learn):
    if transf_learn:
        transf = transforms.Compose([
            transforms.ToTensor()
        ])
    else:
        transf = transforms.Compose([
            HistEq(),
            AddChannel(),
            transforms.ToTensor()
        ])
        print(transf(image))  

    return transf(image).to(torch.float32).unsqueeze_(0)


In [12]:
#evalution stats

In [13]:
# create outer class
class Stats:
	def __init__(self,framework,gender):
		self.framework = framework
		self.gender = gender
		self.probability = 0
		self.picAnalysed = 0
		self.age = 0
        

	# create a 1st Inner class
	class Race:
		def __init__(self):
			self.white = 0
			self.black = 0
			self.asian = 0
			self.indian = 0
			self.unknown = 0


	# create a 2nd Inner class
	class Emotion:
		def __init__(self):
			self.angry = 0
			self.disgust = 0
			self.fear = 0
			self.happy = 0
			self.sad = 0
			self.suprise = 0
			self.neutral = 0

In [14]:
def printDemoClassiOutput(res1,res2):
    print("Democlassi comparision")
    print("---Democlassi---")
    print("Age:---",{round(res2[0], 0)},"---")
    print("Gender:---",{res2[2]},"---")
    print("Race:---",{res2[4]},"---")
    print("Emotion:---",{res1[1]},"---") #res1 for emotion

In [15]:
def deepfaceStats(stats,obj,prob):
    stats.age = stats.age + obj['age']
    stats.probability = stats.probability + prob
    stats.picAnalysed= stats.picAnalysed + 1 
    
    if obj['dominant_race'] == "white":
        stats.race.white = stats.race.white+1
    elif obj['dominant_race']  == "black":
        stats.race.black = stats.race.black+1
    elif obj['dominant_race']  == "asian":
        stats.race.asian = stats.race.asian+1
    elif obj['dominant_race']  == "indian":
        stats.race.indian = stats.race.indian+1
    else:
        stats.race.unknown = stats.race.unknown+1
        
        
    if obj['dominant_emotion']== "angry":
        stats.emotion.angry= stats.emotion.angry+1
    elif obj ['dominant_emotion'] == "disgust":
        stats.emotion.disgust = stats.emotion.disgust+1
    elif obj['dominant_emotion']== "afraid":
        stats.emotion.afraid = stats.emotion.afraid+1
    elif obj['dominant_emotion']== "happy":
        stats.emotion.happy = stats.emotion.happy+1
    elif obj['dominant_emotion'] == "sad":
        stats.emotion.sad = stats.emotion.sad+1
    elif obj['dominant_emotion'] == "surprised":
        stats.emotion.suprise = stats.emotion.suprise+1
    else:
        stats.emotion.neutral = stats.emotion.neutral+1

In [16]:
def classiStats(stats,res1,res2,prob):
    stats.age = stats.age + round(res2[0],0)
    stats.probability = stats.probability + prob
    stats.picAnalysed = stats.picAnalysed + 1 
    
    if res2[4] == "White":
        stats.race.white = stats.race.white+1
    elif res2[4] == "Black":
        stats.race.black = stats.race.black+1
    elif res2[4] == "Asian":
        stats.race.asian = stats.race.asian+1
    elif res2[4] == "Indian":
        stats.race.indian = stats.race.indian+1
    else:
        stats.race.unknown = stats.race.unknown+1
        
        
    if res1[1] == "Angry":
        stats.emotion.angry= stats.emotion.angry+1
    elif res1[1] == "Disgusted":
        stats.emotion.disgust = stats.emotion.disgust+1
    elif res1[1] == "Afraid":
        stats.emotion.afraid = stats.emotion.afraid+1
    elif res1[1] == "Happy":
        stats.emotion.happy = stats.emotion.happy+1
    elif res1[1] == "Sad":
        stats.emotion.sad = stats.emotion.sad+1
    elif res1[1] == "Surprised":
        stats.emotion.suprise = stats.emotion.suprise+1
    else:
        stats.emotion.neutral = stats.emotion.neutral+1

In [17]:
def evaluateDemoclassi(folder_dir, save_dir,statsmen,statswomen):
    for images in os.listdir(folder_dir):
        if (images.endswith(".png") or images.endswith(".jpg")or images.endswith(".jpeg")):
            new=folder_dir+"/"+images
            print(new)
            try: 
                image = face_recognition.load_image_file(new)
                face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="resnet")
                if len(face_locations) != 0 :
                    nofpeople=len(face_locations)
                    for face_location in face_locations:
                        top, right, bottom, left = face_location
                        face_image = image[top:bottom,left:right]
                        pil_img = Image.fromarray(face_image)
                        pil_image = pil_img.resize((400,400),Image.Resampling.LANCZOS)
                        res1, res2 = democlassiEvaluate(pil_image)

                        #probability of the person 
                        if nofpeople == 1 :
                            prob = 1
                        else:
                            prob = 1/nofpeople

                        if res2[2].lower() == "man" :
                            classiStats(statsmen,res1,res2,prob)
                        else:
                            classiStats(statswomen,res1,res2,prob)
                        #democlassiOutput(res1,res2)
            except:
                continue

In [18]:
def evaluateDeepface(folder_dir, save_dir,deepstatsMen,deepstatsWomen):
    for images in os.listdir(folder_dir):
        if (images.endswith(".png") or images.endswith(".jpg")or images.endswith(".jpeg")):
            new=folder_dir+"/"+images
            print(new)
            try: 
                image = face_recognition.load_image_file(new)
                face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="resnet")
                if len(face_locations) != 0 :
                    nofpeople=len(face_locations)
                    print("Number of people: "+str(nofpeople))
                    for face_location in face_locations:
                        top, right, bottom, left = face_location
                        face_image = image[top:bottom,left:right]
                        pil_img = Image.fromarray(face_image)
                        pil_image = pil_img.resize((400,400),Image.Resampling.LANCZOS)

                        fileLocation=save_dir+'/'+'tmp.jpg' #temporary saves the data
                        pil_image.save(fileLocation,"JPEG")
                        obj= DeepFace.analyze(img_path=fileLocation, enforce_detection=False)

                        #probability of the person 
                        prob = 1/nofpeople


                        if obj["gender"] == "Man" :
                            deepfaceStats(deepstatsMen,obj,prob)
                        else:
                            deepfaceStats(deepstatsWomen,obj,prob)

                        deepfaceOutput(obj)
            except: 
                continue
                        
                    #printDemoClassiOutput(obj)
    #print('++++++++++++++++++++++++++++++++++++++++++')  

In [19]:
def printStats(stats):
    if stats.picAnalysed  != 0 :
        print(" Number of image analyzed: " + str(stats.picAnalysed))
        averageAge = stats.age / stats.picAnalysed 
        print("average age:" + str(averageAge))
        print("probability:  " + str(stats.probability ))


        print("-----------------Race------------------")
        print("White: "+ str(stats.race.white))
        print("Black: " + str(stats.race.black))
        print("Asian:" +str(stats.race.asian))
        print("Indian: " + str(stats.race.indian))
        print("Unknown: " + str(stats.race.unknown))

        print("---------Emotion------------------")
        print("Angry: " + str(stats.emotion.angry))
        print("Disgust: " + str(stats.emotion.disgust))
        print("fear: " + str(stats.emotion.fear))
        print("Hppy: " + str(stats.emotion.happy))
        print("sad: " + str(stats.emotion.sad))
        print("Suprise:" + str(stats.emotion.suprise))
        print("neutral: " + str(stats.emotion.neutral) )
    else: 
        print("no stats for detected"+stats.framework +" for "+ stats.gender)




In [20]:
import time


dataset_dir = "/home/manish/Documents/dataset"
save_dir = "/home/manish/Documents/police"
tmp_dir = "/home/manish/Documents" # directory to store temporary created image


wordSearch=[" ceo "," CEO ","-ceo-","-CEO-","CHIEF EXECUTIVE OFFICER","chief executive officer"]

wordSearch2=[" police ","-police-","/police-"]
index = 0


for dataset in os.listdir(dataset_dir):
 
    
    datasetName = dataset_dir+"/"+dataset
    print("datasetName: "+ datasetName)
    data = pd.read_parquet(datasetName, engine='pyarrow')
    
    dfn= data.to_numpy()
    rows = dfn.shape[0]
    cols = dfn.shape[1]
    val = -1 # inital value no word match
    
    
    
    for x in range(0, rows-1): #for the end of the dataset 
        
        if index == 100:
            break
        for s in wordSearch2:
            val=dfn[x][1].lower().find(s)
            if val != -1 :  #val takes -1 if the word is not encountered
                break
            
        if val != -1:  # val has found the word
            url = dfn[x][0]
            print("url: "+url)
            print("+--------------------------------------+")
            print("Text: "+dfn[x][1])
            try:
                request=urllib.request.Request(url,None,headers) #The assembled request
                #resource = urllib.request.urlopen(request,context=ssl.create_default_context(cafile=certifi.where()))
                resource = urllib.request.urlopen(request,context=context)
                download_image_link= save_dir + "/" + "image"+ str(index) +".jpg"
                output = open(download_image_link,"wb")
                output.write(resource.read())
                output.close()
                index = index+1
            except:
                continue      
    

demostatsMen = Stats("democlassi","men")
demostatsMen.race = Stats.Race()
demostatsMen.emotion = Stats.Emotion()

demostatsWomen= Stats("democlassi","women")
demostatsWomen.race = Stats.Race()
demostatsWomen.emotion = Stats.Emotion()

##deepface statstics evaluation
deepstatsMen = Stats("deepface","men")
deepstatsMen.race = Stats.Race()
deepstatsMen.emotion = Stats.Emotion()

deepstatsWomen= Stats("deepface","women")
deepstatsWomen.race = Stats.Race()
deepstatsWomen.emotion = Stats.Emotion()



st = time.time()
evaluateDemoclassi(save_dir,tmp_dir,demostatsMen,demostatsWomen)
et = time.time()


printStats(demostatsMen)
printStats(demostatsWomen)

print("Time taken by democlassi  is : "+ str(round(et-st)) + "seconds")


st = time.time()
evaluateDeepface(save_dir,tmp_dir,deepstatsMen,deepstatsWomen)
et = time.time()

printStats(deepstatsMen)
printStats(deepstatsWomen)
print("Time taken by deepface  is : "+str(round(et-st))+ "seconds")



datasetName: /home/manish/Documents/dataset/part-00039-5d6701c4-b238-4c0a-84e4-fe8e9daea963-c000.snappy.parquet
url: https://nampapolice.org/ImageRepository/Document?documentID=7689
+--------------------------------------+
Text: Several Nampa Police Officers in uniform in front of the Nampa Police Station and the words press re
url: https://portal-images.azureedge.net/auctions-2021/dix-no10154/images/9acd0fe3-89b8-40fc-b8fd-ad3d00f513e1.jpg
+--------------------------------------+
Text: A small Collection of Scottish Police Badges
url: https://www.dailyherald.com/storyimage/DA/20131126/news/711269819/EP/1/1/EP-711269819.jpg&amp;updated=201311261625&amp;MaxW=800&amp;maxH=800&amp;noborder
+--------------------------------------+
Text: Bob Chwedyk/bchwedyk@dailyherald.comFremont Township Highway Commissioner Bill Grinnell, right, is helped by Jose Mendoza of Elly's Pancake House to load turkeys into his vehicle for the Fremont Township Food Pantry. Fremont Township joined the Mundelein Po

url: https://media.pri.org/s3fs-public/story/images/RTX13TNN.jpg
+--------------------------------------+
Text: A Kenyan police officer tries to secure an area inside the Westgate Shopping Centre after a militant siege.
url: https://collections.ushmm.org/iiif-b/assets/761581
+--------------------------------------+
Text: Civilians arrested by German police are held prisoner in the courtyard of the Montelupich prison in Krakow.
url: https://assets.telegraphindia.com/telegraph/9075993c-12f6-4cba-8440-0b4c0291b8a7.jpg
+--------------------------------------+
Text: A police officer stops Rahul Gandhi and Priyanka Gandhi Vadra from entering Meerut on Tuesday. Barred by Uttar Pradesh police, Rahul and Priyanka talked to the families on the phone and returned to Delhi.
url: https://vid.alarabiya.net/images/2018/03/21/d21723d7-9b05-42e3-975b-7874751239f1/d21723d7-9b05-42e3-975b-7874751239f1.jpg
+--------------------------------------+
Text: Indian para-military force soldiers push exiled Tibet

url: https://clipartart.com/images/police-officer-face-clipart-5.jpg
+--------------------------------------+
Text: Police officer face clipart clip download Clipart of a Happy Caucasian Male Police Officer Face - Royalty ... clip download
url: https://www.ekathimerini.com/wp-content/uploads/2021/06/Mitsotakis_Volkswagen_Astypalea_reuters.jpg?1622655869147
+--------------------------------------+
Text: Greek Prime Minister Kyriakos Mitsotakis looks at a Volkswagen ID.4 electric car, during a delivery ceremony of service cars and chargers to the police and the port authority of the island of Astypalea, Greece, June 2, 2021. [Alexandros Vlachos/Pool via Reuters]
url: https://www.gannett-cdn.com/presto/2018/09/29/PSAY/112be1fd-0ea4-432d-a7a5-adef6c8ce218-H2Oi-13.jpg?crop=6071,4548,x0,y0&amp;width=80&amp;height=60&amp;fit=bounds
+--------------------------------------+
Text: The streets of Ocean City were full of cars, crowds and a heavy police presence on Friday, Sept 28, 2018. Popular ca

url: https://a57.foxnews.com/a57.foxnews.com/static.foxnews.com/foxnews.com/content/uploads/2018/09/640/320/1862/1048/nigeria_youth_vigilantes.jpg?ve=1&amp;tl=1?ve=1&amp;tl=1
+--------------------------------------+
Text: June 11, 2013: In this photo taken with a mobile phone, vigilante youths pose for a photographs in Maiduguri, Nigeria. As soldiers continue an offensive against radical Islamic extremists in northeast Nigeria, young men armed with machetes and sticks have now entered the streets of the regions biggest city, targeting suspected fighters. The members of the Civilian JTF, a play off the acronym used to describe the joint military and police taskforce in the region, already have come under attack from extremists for pointing out suspects to soldiers. However, it remains unclear what happens to those the group points out, as some suspects in Maiduguri have already been shot dead by security forces and left rotting in ditches in the city.
url: https://d.newsweek.com/en/full

url: https://www.odmp.org/media/image/officer/1087/orig/akins%2C-harry-1962.jpg
+--------------------------------------+
Text: Patrolman Harry H. Akins | Chatham County Police Department, Georgia
url: https://www.ktsm.com/wp-content/uploads/sites/38/2016/11/NMSU20Police20Lights_1479220487348_12900547_ver1.0.jpg?w=1920&h=1080&crop=1
+--------------------------------------+
Text: NMSU Police Lights_1479220487348.jpg
url: https://i.pinimg.com/originals/10/63/96/106396f770d71fe58a4913ef56e84d5e.jpg
+--------------------------------------+
Text: Children S Road Carpet Kids Cars Carpet Police Fire Station 3mt X
url: https://www.parkcity.org/Home/ShowImage?id=38710&t=636717411617070000
+--------------------------------------+
Text: Mayor Beerman Swearing in New Police Officers
url: https://im-media.voltron.voanews.com/Drupal/01live-166/styles/892x501/s3/2019-04/93DC5FF4-23DD-47B7-9241-D365F6D10859.jpg?itok=MM6kXvhJ
+--------------------------------------+
Text: Police officers search document

...

Action: race:  75%|███████▌  | 3/4 [00:01<00:00,  1.47it/s]  

1/1 [==============================] - 1s 787ms/step


Action: race: 100%|██████████| 4/4 [00:02<00:00,  1.48it/s]

Deepface comparision
----Deepface---
Age:--- {26} ---
Gender:--- {'Man'} ---
Race:--- {'black'} ---
Emotion:--- {'happy'} ---
------------------------------------------- 
/home/manish/Documents/police/image79.jpg


 Number of image analyzed: 72
average age:30.27777777777778
probability:  33.45000000000001
-----------------Race------------------
White: 32
Black: 18
Asian:9
Indian: 6
Unknown: 7
---------Emotion------------------
Angry: 8
Disgust: 0
fear: 0
Hppy: 17
sad: 8
Suprise:0
neutral: 39
 Number of image analyzed: 8
average age:30.125
probability:  3.5500000000000003
-----------------Race------------------
White: 6
Black: 0
Asian:0
Indian: 0
Unknown: 2
---------Emotion------------------
Angry: 1
Disgust: 0
fear: 0
Hppy: 6
sad: 0
Suprise:0
neutral: 1
Time taken by deepface  is : 356seconds
